### CMPN403 Programming Assignment
#### Regex to Minimized DFA Conversion
Team:
- Ahmed Tarek Abdellatif 1190157
- Mostafa Mohammad Mahmoud El-Nashar 1190212

### TODOS:
- #### NFA2DFA(FSM)-> FSM (Decide whether the current FSM datastructures for states are efficient and useable)
- #### MinimizeDFA(FSM)-> FSM

#### State and FSM Classes
##### FSM reads/writes to json, performs Thompson's algorithm on given string

In [65]:
import json
# from typing import Optional


class State:
    def __init__(self, name: str = "", transitions: dict[str, list[str]] = None):
        self.name: str = name
        # {input : [list of states]}
        self.transitions: dict[str, list[str]] = transitions if transitions is not None else dict()

    def __iter__(self):
        for key, value in self.transitions.items():
            yield (key, value)


class FSM:

    stateCounter = -1

    @staticmethod
    def __getNewStateName():
        FSM.stateCounter += 1
        return "Q" + str(FSM.stateCounter).zfill(3)

    @staticmethod
    def resetStateCounter():
        FSM.stateCounter = -1

    def __init__(self, literal: str = None, single_char: str = None):
        """
        *** Doesn't validate the literal string ***\n
        """
        self.nonTerminalStates: dict[str, State] = dict()  # {state_name : state}
        self.terminalStates: dict[str, State] = dict()  # {state_name : state}
        self.startingState: State = None
        if literal is not None: self.__literal_to_fsm(literal)
        elif single_char is not None: self.__single_char_to_fsm(single_char)

    def __literal_to_fsm(self, literal: str):
        """
        *** Doesn't validate the literal string ***\n
        Intializes the FSM to describe a literal string
        """
        if literal == "":
            s = State(FSM.__getNewStateName())
            self.startingState = s
            self.terminalStates[s.name] = s
            return
        # Non-empty string
        older_state_name = starting_state_name = FSM.__getNewStateName()
        for char in literal:
            new_state_name = FSM.__getNewStateName()
            self.nonTerminalStates[older_state_name] = State(older_state_name, {char: [new_state_name]})
            older_state_name = new_state_name
        self.terminalStates[older_state_name] = State(older_state_name)
        self.startingState = self.nonTerminalStates[starting_state_name]

    def __single_char_to_fsm(self, single_char: str):
        """
        Intializes the FSM to describe a single character
        """
        s = State(FSM.__getNewStateName(), {single_char: [FSM.__getNewStateName()]})
        self.nonTerminalStates[s.name] = s
        self.startingState = s
        self.terminalStates[s.transitions[single_char][0]] = State(s.transitions[single_char][0])

    @staticmethod
    def read_JSON(JSONFILE: str) -> "FSM":
        """
        Takes JSON file name as string and reads an fsm from it
        """
        fsm = FSM()
        with open(JSONFILE) as f:
            file_contents = f.read()
            parsed_json: dict[str, str | dict[str, bool | dict[str, list[str]]]] = json.loads(file_contents)
            starting_state_name = parsed_json["@startingState"]
            parsed_json.pop("@startingState")
            for name, transitions in parsed_json.items():
                terminal = transitions["@isTerminatingState"]
                transitions.pop("@isTerminatingState")
                s = State(name, transitions)
                if name == starting_state_name:
                    fsm.startingState = s
                if terminal:
                    fsm.terminalStates[name] = s
                else:
                    fsm.nonTerminalStates[name] = s
        return fsm

    def write_JSON(self, JSONFILE: str):
        """
        Takes JSON file name as string and writes the fsm to it
        """
        with open(JSONFILE, "w") as f:
            data = {'@startingState': self.startingState.name}
            data.update({s.name: {"@isTerminatingState": False, **dict(s)} for s in self.nonTerminalStates.values()})
            data.update({s.name: {"@isTerminatingState": True, **dict(s)} for s in self.terminalStates.values()})
            json.dump(data, f, indent=4, sort_keys=True)

    def Concatenate(self, fsm: "FSM") -> "FSM":
        """
        Concatenates the current FSM with another FSM ***in place***\n
        i.e ( self . fsm )
        """
        past_terminal_states = self.terminalStates
        # Adding the old terminal states to the non terminal states
        self.nonTerminalStates.update(past_terminal_states)
        # Adding the new non terminal states to the non terminal states
        self.nonTerminalStates.update(fsm.nonTerminalStates)
        # Updating the terminal states to the new ones
        self.terminalStates = fsm.terminalStates
        for state in past_terminal_states.values():
            if state.transitions.get("ε") is None:
                state.transitions["ε"] = [fsm.startingState.name]
            else:
                self.transitions["ε"].append(fsm.startingState.name)
        return self

    @staticmethod
    def Alternate(fsms: list["FSM"]) -> "FSM":
        """
        Alternates a list of FSMs ***in place***\n
        i.e ( fsms[0] | fsms[1] | ... | fsms[n] )
        """
        if len(fsms) == 1:
            return fsms[0]
        result_fsm = FSM()
        result_fsm.startingState = State(FSM.__getNewStateName())
        result_fsm.nonTerminalStates.update({result_fsm.startingState.name: result_fsm.startingState})
        new_terminating_state = State(FSM.__getNewStateName())
        result_fsm.terminalStates.update({new_terminating_state.name: new_terminating_state})

        # Mapping ε moves from new starting state to all the old starting states
        result_fsm.startingState.transitions["ε"] = [fsm.startingState.name for fsm in fsms]

        for fsm in fsms:
            # Adding the all old states to the new non terminal states
            result_fsm.nonTerminalStates.update(fsm.nonTerminalStates)
            result_fsm.nonTerminalStates.update(fsm.terminalStates)
            # Mapping epislon moves from old terminal states to new terminal state
            for state in fsm.terminalStates.values():
                if not state.transitions.get("ε"):
                    state.transitions["ε"] = [new_terminating_state.name]
                else:
                    state.transitions["ε"].append(new_terminating_state.name)
        return result_fsm

    def Zero_Or_More(self) -> "FSM":
        """
        Applies Kleene star to the current FSM ***in place***\n
        i.e ( self* )
        """
        new_starting_state = State(FSM.__getNewStateName())
        new_terminating_state = State(FSM.__getNewStateName())

        # Mapping epislon moves from new starting state to old starting state and new terminating state
        new_starting_state.transitions["ε"] = [self.startingState.name, new_terminating_state.name]

        past_terminal_states = self.terminalStates
        # Adding the old terminal states to the non terminal states
        self.nonTerminalStates.update(past_terminal_states)
        self.nonTerminalStates[new_starting_state.name] = new_starting_state
        # Updating the starting state to the new one
        self.startingState = new_starting_state
        # Updating the terminal states to the new one
        self.terminalStates = {new_terminating_state.name: new_terminating_state}

        # Mapping epislon moves from old terminal states to new terminal state and new starting state
        for state in past_terminal_states.values():
            if not state.transitions.get("ε"):
                state.transitions["ε"] = [new_terminating_state.name, self.startingState.name]
            else:
                state.transitions["ε"].extend([new_terminating_state.name, self.startingState.name])
        return self

    def One_Or_More(self) -> "FSM":
        """
        Applies Kleene plus to the current FSM ***in place***\n
        i.e ( self+ )
        """
        new_starting_state = State(FSM.__getNewStateName())
        new_terminating_state = State(FSM.__getNewStateName())

        # Mapping epislon moves from new starting state to old starting state
        new_starting_state.transitions["ε"] = [self.startingState.name]

        past_terminal_states = self.terminalStates
        # Adding the old terminal states to the non terminal states
        self.nonTerminalStates.update(past_terminal_states)
        self.nonTerminalStates[new_starting_state.name] = new_starting_state
        # Updating the starting state to the new one
        self.startingState = new_starting_state
        # Updating the terminal states to the new one
        self.terminalStates = {new_terminating_state.name: new_terminating_state}

        # Mapping epislon moves from old terminal states to new terminal state and new starting state
        for state in past_terminal_states.values():
            if not state.transitions.get("ε"):
                state.transitions["ε"] = [new_terminating_state.name, self.startingState.name]
            else:
                state.transitions["ε"].extend([new_terminating_state.name, self.startingState.name])
        return self

    def Optional(self) -> "FSM":
        """
        Applies Kleene question mark to the current FSM ***in place***\n
        i.e ( self? )
        """
        new_starting_state = State(FSM.__getNewStateName())
        new_terminating_state = State(FSM.__getNewStateName())

        # Mapping epislon moves from new starting state to old starting state and new terminating state
        new_starting_state.transitions["ε"] = [self.startingState.name, new_terminating_state.name]

        past_terminal_states = self.terminalStates
        # Adding the old terminal states to the non terminal states
        self.nonTerminalStates.update(past_terminal_states)
        self.nonTerminalStates[new_starting_state.name] = new_starting_state
        # Updating the starting state to the new one
        self.startingState = new_starting_state
        # Updating the terminal states to the new one
        self.terminalStates = {new_terminating_state.name: new_terminating_state}

        # Mapping epislon moves from old terminal states to new terminal state
        for state in past_terminal_states.values():
            if not state.transitions.get("ε"):
                state.transitions["ε"] = [new_terminating_state.name]
            else:
                state.transitions["ε"].extend([new_terminating_state.name])
        return self


Testing FSM class

In [2]:
# FSM('ab').Alternate(FSM('cd')).write_JSON('ab or cd.json')
# FSM('ab').Concatenate(FSM('cd')).write_JSON('abcd.json')
# FSM('ab').Zero_Or_More().write_JSON('abstar.json')
# FSM('ab').One_Or_More().write_JSON('abplus.json')
# FSM('ab').Optional().write_JSON('aboptional.json')
# FSM('ab').Alternate(FSM('cd')).Zero_Or_More().write_JSON('_ab or cd_star.json')
# FSM.read_JSON("ab or cd.json").write_JSON('.json')

#### Regex to NFA

In [66]:
def Regex2NFA(regex: str) -> "FSM":
    """
    Converts a regex string to an NFA
    """
    i = 0
    # e.g: "ac(b|d)|e*" -> ['a', 'c', '(', 'b', '|', 'd', ')', '|', 'e', '*']
    regex : list[str] = [*regex]

    # Compute the NFA for each bracketed expression or character class
    # e.g:['a', 'c', '(', 'b', '|', 'd', ')', '|', 'e', '*']  -> ['a', 'c', FSM, '|', 'e', '*']
    while i < len(regex):
        if regex[i] == '(':
            open_bracket_index = i
            bracket_depth = 1
            while bracket_depth > 0:
                i += 1
                if regex[i] == '(':
                    bracket_depth += 1
                elif regex[i] == ')':
                    bracket_depth -= 1
            modifying_fsm = Regex2NFA(regex[open_bracket_index + 1:i])
            regex[open_bracket_index:i + 1] = [modifying_fsm]
            i = open_bracket_index
        if regex[i] == '[':
            open_bracket_index = i
            while regex[i] != ']':
                i += 1
            modifying_fsm = FSM(single_char = ''.join(regex[open_bracket_index + 1:i]))
            regex[open_bracket_index:i + 1] = [modifying_fsm]
            i = open_bracket_index
        i += 1

    # Apply the unary operators
    # e.g: ['a', 'c', FSM, '|', 'e', '*'] -> ['a', 'c', FSM, '|', FSM]
    i = 0
    regex : list[str | "FSM"]
    while i < len(regex):
        if regex[i] in ['*', '+', '?']:
            if not isinstance(regex[i - 1], FSM):
                regex[i - 1] = FSM(single_char = regex[i - 1])
            if regex[i] == '*':
                regex[i - 1] = regex[i - 1].Zero_Or_More()
            elif regex[i] == '+':
                regex[i - 1] = regex[i - 1].One_Or_More()
            elif regex[i] == '?':
                regex[i - 1] = regex[i - 1].Optional()
            del regex[i]
            continue
        i += 1

    # Join contiguous literals to fsms
    # e.g: ['a', 'c', FSM, '|', FSM] -> [FSM, FSM, '|', FSM]
    i = 0
    starting_char_index = 0
    while i < len(regex):
        if isinstance(regex[i], FSM) or regex[i]  == '|':
            if i == starting_char_index:
                starting_char_index += 1
                i += 1
            else:
                regex[starting_char_index:i] = [FSM(''.join(regex[starting_char_index:i]))]
                starting_char_index += 2
                i = starting_char_index
        elif i == len(regex) - 1:
            regex[starting_char_index:] = [FSM(''.join(regex[starting_char_index:]))]
            break
        else:
            i += 1

    # Concatenate the FSMs within the alternatives
    # e.g: [FSM, FSM, '|', FSM] -> [FSM, FSM]
    i = 0
    starting_fsm_index = 0
    fsms_to_alternate : list['FSM'] = []
    while i < len(regex):
        if regex[i] == '|':
            result_fsm = regex[starting_fsm_index]
            for fsm in regex[starting_fsm_index + 1:i]:
                result_fsm = result_fsm.Concatenate(fsm)
            fsms_to_alternate.append(result_fsm)
            starting_fsm_index = i + 1
        elif i == len(regex) - 1:
            result_fsm = regex[starting_fsm_index]
            for fsm in regex[starting_fsm_index + 1:]:
                result_fsm = result_fsm.Concatenate(fsm)
            fsms_to_alternate.append(result_fsm)
        i += 1

    # Apply the alternation
    return FSM.Alternate(fsms_to_alternate)

#### Export FSM to PNG

In [67]:
import graphviz
FSM.resetStateCounter()
f =  Regex2NFA('(a|b)*abb+d?(a?|(cb)*)+')
f.write_JSON('test.json')

def addTransitions(fsm: "FSM", dot, state: State, visited : set[str]):
    for key, value in state.transitions.items():
        for v in value:
            dot.edge(state.name, v, label=key)
            if v not in visited:
                visited.add(v)
                if v in fsm.nonTerminalStates:
                    addTransitions(fsm, dot, fsm.nonTerminalStates[v], visited)
                else:
                    addTransitions(fsm, dot, fsm.terminalStates[v], visited)

def create_graph(fsm : "FSM", label, filename):
    dot = graphviz.Digraph('NFA', filename=filename,format='png')
    dot.node_attr['shape'] = 'circle'
    dot.node('', shape='none')

    for state in fsm.nonTerminalStates.values():
        dot.node(state.name, state.name)

    for state in fsm.terminalStates.values():
        dot.node(state.name, state.name, shape='doublecircle')

    dot.edge('', fsm.startingState.name, label='start')
    addTransitions(fsm, dot, fsm.startingState, {fsm.startingState.name})

    dot.graph_attr['label'] = label
    dot.graph_attr['rankdir'] ='LR'
    dot.graph_attr['overlap'] = 'false'
    dot.graph_attr['splines'] = 'true'
    dot.graph_attr['sep'] = '+1'
    dot.graph_attr['nodesep'] = '1'
    dot.graph_attr['ranksep'] = '0.7 equally'
    dot.render(cleanup=True)

create_graph(f, 'NFA', 'NFA')

#### Verify Regex

In [ ]:
import re


def verify_regex(regex):
    try:
        re.compile(regex)
    except re.error:
        print("regex Rejected")
    else:
        if re.search(r"(^\/{1,})|(\/{2})$", regex):
            print("regex rejected")
        else:
            print("SUCCESS")
            # todo : C'MON do something


x = input("Enter a regex to verify : ")
verify_regex(x)


#### Interpreting ranges 😲🤓

In [42]:
# test_str = "A-Za-z0-9._&+@"
# test_str = "a-zA-Z0-9_$-"
test_str = "a-zA-Z0-9-_"
 
# printing original string
print("The original string is : " + test_str)
 
# Convert String ranges to list
# Using for loop and string manipulation
def listFromRange(rng : str) -> list[str]:
    res = []
    for i in range(0, len(rng), 3):
        s = rng[i:i+3]
        if s[1] == '-' and (s[0::2].isdecimal() or s[0::2].isalpha()):
            start, end = map(int, [ord(char) for char in s.split('-')])
            res.append(chr(start))
            for i in range(start+1, end+1):
                res.append(chr(i))
        else:
            if len(s) == 1:
                res.append(s)
            else:
                res.extend([*s])
    return res
# printing result
print("List after conversion from string : " + str(listFromRange(test_str)))

The original string is : a-zA-Z0-9-_
List after conversion from string : ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '_']
